Carga de librerías necesarias

In [1]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPRegressor
import bokeh.plotting as bpl

In [2]:
bpl.output_notebook()

Loading BokehJS ...

Definición de funciones necesarias para entrenar el modelo de alcance

In [3]:
def entrenador(arreglo,alcances):
    """Entrena el modelo utilizando un arreglo de publicaciones o un dataframe y sus alcances.
    
    Parameters:
        arreglo (arreglo de numpy, también puede ser un dataframe):
            Arreglo multidimensional con los valores de las métricas para cada publicación.
            Cada publicación está en una fila del arreglo.
            El orden de las métricas debe ser el siguiente [likes,love,angry,wow,haha,sad,shares].
        alcances (arreglo de numpy, tambien puede ser una serie):
            Arreglo unidimensional con los valores de los alcances para cada publicación.
            Cada publicación está en una fila del arreglo.
        
    Returns:
        red (red neuronal de Sklearn):
            Modelo de red neuronal entrenada para predecir los alcances de publicaciones.
            
    """
    logtrain = np.log1p(arreglo)
    logpredi = np.log1p(alcances)
    
    red = MLPRegressor(alpha=0.01, hidden_layer_sizes = (10,), max_iter = 50000, 
                 activation = 'logistic', learning_rate = 'adaptive',solver= 'lbfgs')
    
    red.fit(logtrain,logpredi)
    
    return red

In [4]:
def predictor(arreglo,modelo):
    """Predice los alcances para un arreglo de publicaciones o un dataframe.
    
    Parameters:
        arreglo (arreglo de numpy, también puede ser un dataframe):
            Arreglo multidimensional con los valores de las métricas para cada publicación.
            Cada publicación está en una fila del arreglo.
            El orden de las métricas debe ser el siguiente [likes,love,angry,wow,haha,sad,shares].
            
        modelo (modelo de sklearn):
            El modelo de predicción entrenado previamente
            
    Returns:
        alcances (arreglo de numpy):
            Arreglo con los alcances para cada publicación.
            
    """
    logdata = np.log1p(arreglo)
    predata = modelo.predict(logdata)
    bacdata = np.expm1(predata)
    
    return bacdata

Carga y procesado de datos para entrenar la red para predecir alcance

In [5]:
data = pd.read_csv("../data/originales/posts.csv")

In [6]:
metricas = ['likes', 'love', 'angry', 'wow', 'haha', 'sad', 'shares']

In [7]:
data["reacciones"] = data[metricas].sum(1)

In [8]:
fdata=data[(data["scope"]!=0)&(data["reacciones"]>10)&(data["reacciones"]<=data["scope"])]

In [9]:
mdata = fdata[metricas + ["scope"]]

In [10]:
mdata[:3]

,likes,love,angry,wow,haha,sad,shares,scope
0,18,7,0,0,0,0,4,3660
1,526,117,189,15,51,8,107,77468
2,28,1,0,0,0,0,13,4399


In [11]:
arr_metricas = mdata[metricas].values

In [12]:
arr_metricas

array([[ 18,   7,   0, ...,   0,   0,   4],
       [526, 117, 189, ...,  51,   8, 107],
       [ 28,   1,   0, ...,   0,   0,  13],
       ...,
       [ 23,   0,   1, ...,   0,   3,   1],
       [ 13,   0,   4, ...,   0,   0,   0],
       [ 24,   0,   0, ...,   0,   0,   0]], dtype=int64)

In [13]:
arr_alcances = mdata["scope"].values

Entrenado de la red predictora de alcance

In [14]:
red = entrenador(arr_metricas,arr_alcances)

Carga de datos de publicaciones y temas

In [44]:
datap = pd.read_csv("../data/originales/PostsViralidadSinSaltos.tsv",sep='\t')
datat = pd.read_csv("../data/originales/TemasViralidadSinSaltos.tsv",sep='\t')

Proceso de datos de temas y publicaciones

In [16]:
datat.columns=[cadena + "_T" for cadena in datat.columns]

In [17]:
datat.rename(columns={"id_T":"idTema"},inplace=True)

Mezcla de los datos de publicaciones y de temas

In [18]:
datamix = pd.merge(datap,datat,how="left",on="idTema")

Llenado de datos vacíos a cero

In [19]:
datafp = datamix[metricas].fillna(0)

Predicción de alcances para todas las publicaciones

In [20]:
datapv = datafp.values

In [21]:
predicciones = predictor(datapv,red)

In [22]:
predicciones

array([   136.4981204 ,    136.4981204 , 119264.05228629, ...,
          136.4981204 ,    136.4981204 ,    136.4981204 ])

Conversión a DataFrame e incorporación al frame general

In [23]:
prediccion = pd.DataFrame(predicciones,columns=["Alcance_estimado"],index=datafp.index)

In [24]:
datamix["Alcance_estimado"] = prediccion

In [25]:
datamix["reacciones"] = datamix[metricas].sum(1)

Definición de Estado a calificar y parámetros para el modelo, alcmax es el alcance máximo posible, es decir la población total a alcanzar
pubmax es el número máximo de publicaciones para fijar el tope de la calificación, usualmente 100 0 200 funciona bien, dependiendo de la ciudad.

In [26]:
estado = ["Ciudad de México"]
alcmax = 8500000
pubmax = 200

Filtrado de las publicaciones a aquellas que están en el estado y selección de columnas importantes

In [27]:
data_filt=datamix[(datamix["estado_T"].isin(estado))][metricas+["score_T","Alcance_estimado","reacciones","idTema","nombre_T"]]

Sustitución de los alcances mayores al alcance máximo por el alcance máximo

In [28]:
data_filt["Alcance_est_top"] = data_filt["Alcance_estimado"].apply(lambda x: min(x,alcmax))

Definición de la función para calcular el alcance extra al alcance de la publicación con mayor alcance para cada tema

In [29]:
def alcance_extra(serie,atope):
    nserie = serie.apply(lambda x: min(x,atope))
    serie_s = nserie.sort_values(ascending=False)
    index = serie_s.index
    a_max  = min(atope,serie_s.max())
    r = (atope - a_max)/atope
    rango = pd.Series(pd.RangeIndex(0,len(serie_s)),index=index)
    mults = np.power(r,rango)
    return (serie_s*mults).sum()-a_max

Agrupación de publicaciones por tema

In [30]:
grupos = data_filt.groupby(["idTema","nombre_T"])

Cálculo del alcance extra para cada tema

In [31]:
por_tema = grupos.apply(lambda x: alcance_extra(x["Alcance_est_top"],alcmax)).to_frame("Alcance_extra")

Cálculos de alcances máximos, suma de alcances y número de publicaciones para cada tema

In [32]:
por_tema["Alcance_max_top"] = grupos["Alcance_est_top"].max()
por_tema["Alcance_suma"] = por_tema["Alcance_max_top"] + por_tema["Alcance_extra"]
por_tema["Publicaciones"] = grupos["Alcance_estimado"].size()

Definición de la función para escalar los valores y calcular las calificaciones 

In [33]:
def scale(inp_domain,out_range,valor):
    valor_estimado = ((out_range[1]-out_range[0])*(valor-inp_domain[0])/(inp_domain[1]-inp_domain[0])) + out_range[0]
    return np.clip(valor_estimado,out_range[0],out_range[1])

Definición de pesos para las calificaciones, deben de sumar a 100

In [34]:
peso_alcance = 50
peso_temas = 50

Cálculo de las calificaciones por alcance, por temas y total

In [35]:
por_tema["Cal_alcance"] = scale((0,np.log10(alcmax)),(0,peso_alcance),np.log10(por_tema["Alcance_suma"]))
por_tema["Cal_publicaciones"] = scale((0,np.log10(pubmax)),(0,peso_temas),np.log10(por_tema["Publicaciones"]))
por_tema["Calificacion"] = por_tema["Cal_alcance"] + por_tema["Cal_publicaciones"]

Definición del frame de salida

In [36]:
salida = por_tema[["Alcance_max_top","Alcance_extra","Alcance_suma","Publicaciones","Cal_alcance","Cal_publicaciones","Calificacion"]]

Ordenando de mayor a menor por calificación y alcance total

In [37]:
lista_final = salida.sort_values(["Calificacion","Alcance_suma"],ascending=False)

Exportando el archivo de salida

In [38]:
lista_final.to_csv("../data/procesados/salida.csv")